In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

# Load CSV files
df_one = pd.read_csv("/home/harald_stabbetorp/Master/e-prop_master_nmbu/sound_fet/one_encoded_features.csv")  # Shape: (num_samples, 12800)
df_two = pd.read_csv("/home/harald_stabbetorp/Master/e-prop_master_nmbu/sound_fet/two_encoded_features.csv")  # Shape: (num_samples, 12800)
df_silence = pd.read_csv("/home/harald_stabbetorp/Master/e-prop_master_nmbu/sound_fet/zeros.csv")  # Shape: (num_samples, 12800)

# Reshape to (num_samples, 25, 512)
X_one = df_one.values.reshape(-1, 25, 512)
X_two = df_two.values.reshape(-1, 25, 512)
X_silence = df_silence.values.reshape(-1, 25, 512)

# Define labels: Silence=0, "one"=1, "two"=2
y_silence = np.zeros((X_silence.shape[0], 1))
y_one = np.ones((X_one.shape[0], 1))  # Label "one" as 1
y_two = np.full((X_two.shape[0], 1), 2)  # Label "two" as 2

# Concatenate sequences in alternating order
X_combined = np.vstack((X_silence, X_one, X_silence, X_two))
y_combined = np.vstack((y_silence, y_one, y_silence, y_two))

# Repeat the sequence 10 times to make the dataset longer
X_combined = np.tile(X_combined, (10, 1, 1))  # Repeat X_combined 10 times
y_combined = np.tile(y_combined, (10, 1))     # Repeat y_combined 10 times

# Shuffle the dataset (important)
X_combined, y_combined = shuffle(X_combined, y_combined, random_state=42)

# Split into training and testing (using the same data for both)
X_train, X_test = X_combined, X_combined
y_train, y_test = y_combined, y_combined

print("X_train shape:", X_train.shape)  # (total_samples * 10, 25, 512)
print("y_train shape:", y_train.shape)  # (total_samples * 10, 1)


2025-03-12 14:01:40.455969: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741784500.482329  845799 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741784500.491472  845799 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-12 14:01:40.523009: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


X_train shape: (40, 25, 512)
y_train shape: (40, 1)


In [2]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

# Define model
inputs = Input(shape=(25, 512))
x = LSTM(256, return_sequences=False)(inputs)
x = Dense(128, activation='relu')(x)
x = Dense(3, activation='softmax')(x)  # 3 output classes: Silence, One, Two

model = Model(inputs, x)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)


2025-03-12 14:01:49.054092: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.4062 - loss: 1.3132 - val_accuracy: 0.1250 - val_loss: 1.0634
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 366ms/step - accuracy: 0.5938 - loss: 0.8602 - val_accuracy: 0.1250 - val_loss: 1.0073
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 313ms/step - accuracy: 0.5938 - loss: 0.7664 - val_accuracy: 1.0000 - val_loss: 0.9984
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 354ms/step - accuracy: 1.0000 - loss: 0.6459 - val_accuracy: 1.0000 - val_loss: 0.9778
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 314ms/step - accuracy: 1.0000 - loss: 0.5525 - val_accuracy: 1.0000 - val_loss: 0.9569
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 315ms/step - accuracy: 1.0000 - loss: 0.4966 - val_accuracy: 1.0000 - val_loss: 0.9400
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 283ms/step - accuracy: 1.0000 - loss: 0.4653 - val_accuracy: 1.0000 - val_loss: 0.9267
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step - accuracy: 1.0000 - loss: 0.4459 - val_accuracy: 1.0000 - val_loss: 0.

In [5]:
# Evaluate on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

# Make predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert softmax output to class index

print("Predicted Classes:", y_pred_classes)  # Show first 10 predictions
print("True Classes:", y_test.flatten())  # Show first 10 actual labels


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 1.0000 - loss: 0.0171
Test Accuracy: 1.0000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Predicted Classes: [2 0 2 0 0 0 1 2 2 0 1 1 1 2 0 0 1 0 0 1 1 2 1 1 1 0 0 0 2 2 2 0 0 0 0 0 2
 0 0 0]
True Classes: [2. 0. 2. 0. 0. 0. 1. 2. 2. 0. 1. 1. 1. 2. 0. 0. 1. 0. 0. 1. 1. 2. 1. 1.
 1. 0. 0. 0. 2. 2. 2. 0. 0. 0. 0. 0. 2. 0. 0. 0.]
